In [3]:
from src.model.filterData import filterData
from src.model.riskMatrix import riskMatrix
from src.model.calculateTransformerInformation import calculateTransformerInformation

import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()


In [24]:
codigo_operacional = "INAT7-01"
familia = "Autotransformador"
resultado = calculateTransformerInformation(cursor, codigo_operacional, familia).executar_calculo()

In [37]:
resultado.CodigoCalculo.unique()

array(['BCH_CAP_REATOR', 'BCH_TD_REATOR', 'IE_TR_ATV_GP',
       'IE_TR_ATV_H2OP', 'IE_TR_BCH_CAP', 'IE_TR_BCH_FUGA',
       'IE_TR_BCH_TD', 'REC_MOL_BOLHAS'], dtype=object)

In [33]:
resultado.UltimaAtualizacaoCalculo.unique()

<DatetimeArray>
['2023-05-30 02:04:36.940000', '2023-05-30 02:04:37.977000',
 '2023-05-30 02:04:37.980000', '2023-05-30 02:04:12.367000',
 '2023-05-30 02:04:12.530000', '2023-05-30 02:04:12.533000',
 '2023-05-30 02:04:12.150000', '2023-05-30 02:04:12.523000',
 '2023-05-30 02:04:12.527000', '2023-05-30 02:16:38.993000',
 '2023-05-30 02:16:39.553000', '2023-05-30 02:16:39.557000']
Length: 12, dtype: datetime64[ns]

In [4]:
codigo_operacional = "INAT7-01"
familia = "Autotransformador"
resultado = riskMatrix(cursor, codigo_operacional, familia).processRiskMatrix()

In [7]:
resultado

,CodigoCalculo,ResultadoCalculo,UltimaAtualizacaoCalculo,descricaoInstalacaoEletrica,descricaoEquipamento,RevisaoVariavel,VariavelCodigo,VariavelDescricao,ResultadoVariavel,dataMedicaoVariavel
0,DEN_ATV,31.0,2023-05-30 02:04:37.980,Itacaiúnas,Autotransformador INAT7-01 Fase A,0,TORQUE_DATA,Tempo de operação do motor na última comutação,1.063684e+09,2023-05-30 05:03:44.237
1,NUM_ATV,22.0,2023-05-30 02:04:37.980,Itacaiúnas,Autotransformador INAT7-01 Fase A,0,TORQUE_DATA,Tempo de operação do motor na última comutação,1.063684e+09,2023-05-30 05:03:44.237
2,DEN_BCH,19.0,2023-05-30 02:04:37.980,Itacaiúnas,Autotransformador INAT7-01 Fase A,0,TORQUE_DATA,Tempo de operação do motor na última comutação,1.063684e+09,2023-05-30 05:03:44.237
3,DEN_CMT,16.0,2023-05-30 02:04:37.980,Itacaiúnas,Autotransformador INAT7-01 Fase A,0,TORQUE_DATA,Tempo de operação do motor na última comutação,1.063684e+09,2023-05-30 05:03:44.237
4,NUM_CMT,13.0,2023-05-30 02:04:37.980,Itacaiúnas,Autotransformador INAT7-01 Fase A,0,TORQUE_DATA,Tempo de operação do motor na última comutação,1.063684e+09,2023-05-30 05:03:44.237
...,...,...,...,...,...,...,...,...,...,...
107563,REC_MOL_ESPECIALISTA,0.0,2023-05-30 02:16:39.557,Itacaiúnas,Autotransformador INAT7-01 Reserva,0,CFMUITOALTA2A_ON,Alerta de Corrente de Fuga Muito Alta na Bucha...,0.000000e+00,2023-05-30 02:15:48.227
107564,REC_MOL_ESPECIALISTA,0.0,2023-05-30 02:16:39.557,Itacaiúnas,Autotransformador INAT7-01 Reserva,0,CFMUITOALTA2B_ON,Alerta de Corrente de Fuga Muito Alta na Bucha...,0.000000e+00,2023-05-30 02:15:48.227
107565,REC_MOL_ESPECIALISTA,0.0,2023-05-30 02:16:39.557,Itacaiúnas,Autotransformador INAT7-01 Reserva,0,CFMUITOALTA2C_ON,Alerta de Corrente de Fuga Muito Alta na Bucha...,0.000000e+00,2023-05-30 02:15:48.227
107566,REC_MOL_ESPECIALISTA,0.0,2023-05-30 02:16:39.557,Itacaiúnas,Autotransformador INAT7-01 Reserva,0,CFMUITOALTA3A_ON,Alerta de Corrente de Fuga Muito Alta na Bucha...,0.000000e+00,2023-05-30 02:15:48.227


In [24]:
query = '''
 SELECT DISTINCT
            Calculo.Codigo AS CodigoCalculo
			, Calculo.Descricao
            , ExecucaoCalculoResultado.Resultado AS HealtIndex 
			, ExecucaoCalculoResultado.UltimaAtualizacao AS UltimaAtualizacaoCalculo
            , ie.Descricao AS descricaoInstalacaoEletrica
            , e.Id AS idEquipamento
            , e.Descricao AS descricaoEquipamento
			, Calculo.GrupoCalculoId
			, gc.Descricao
            , f.Nome
        FROM [EngineCalculo].[CalculoResultado]  As ExecucaoCalculoResultado
            INNER JOIN [EngineCalculo].[CalculoResultadoEquipamento] 
                ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId 
                AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
            INNER JOIN [Treetech].[ViewEngine_UltimoJob] 
                ON ViewEngine_UltimoJob.EquipamentoId = CalculoResultadoEquipamento.EquipamentoId 
                AND ViewEngine_UltimoJob.JobId = ExecucaoCalculoResultado.JobId
            INNER JOIN [EngineCalculo].[Calculo] 
                ON Calculo.Id = ExecucaoCalculoResultado.CalculoId
            INNER JOIN [EngineCalculo].[CalculoResultadoVariavel] As EntradaVariavel 
                ON ExecucaoCalculoResultado.JobId = EntradaVariavel.JobId
            INNER JOIN [EngineCalculo].[RevisaoVariavel] As RevisaoVariavel 
                ON RevisaoVariavel.VariavelId = EntradaVariavel.VariavelId 
                AND RevisaoVariavel.Revisao = EntradaVariavel.Revisao
            INNER JOIN [EngineCalculo].[Variavel] 
                ON Variavel.Id = EntradaVariavel.VariavelId 
            INNER JOIN Equipamento AS e 
                ON e.Id = CalculoResultadoEquipamento.EquipamentoId
            INNER JOIN EquipamentoInstalacaoEletrica AS eie 
                ON e.Id = eie.EquipamentoId
            INNER JOIN InstalacaoEletrica AS ie 
                ON  ie.Id = eie.InstalacaoEletricaId
            INNER JOIN Familia AS f 
                ON f.Id = e.FamiliaId
			INNER JOIN EngineCalculo.CategoriaVariavel AS cv 
				ON cv.Id = Variavel.CategoriaVariavelId
			LEFT JOIN EngineCalculo.GrupoCalculo AS gc
				ON gc.Id = Calculo.GrupoCalculoId
        WHERE 1 =1 
		  --f.Nome = 'Transformador de Força'
            AND e.CodigoOperacional = '05T3'
            AND gc.Descricao = 'Health Index do Subsistema'
            AND cv.Descricao LIKE '%Bucha%'
						-- Parte Ativa
						-- Comutador sobcarga
						-- Parte Ativa
						-- Acessórios
						-- Ambiente
						-- Tanque
						-- Resfriamento
						-- Preservação do Óleo Isolante 
						-- Bucha
        ORDER BY ExecucaoCalculoResultado.UltimaAtualizacao DESC
				 , ie.Descricao
				 , e.Descricao
        '''
resultado = cursor.execute(query)

In [25]:

colunas = ['CodigoCalcul', 'Descricao', 'HealtIndex', 'UltimaAtualizacaoCalculo', 'descricaoInstalacaoEletrica', 'idEquipamento', 'descricaoEquipamento', 'GrupoCalculoId', 'Descricao', 'Nome']
dados = [dict(zip(colunas, row)) for row in resultado]
df = pd.DataFrame(dados)

In [26]:
df

,CodigoCalcul,Descricao,HealtIndex,UltimaAtualizacaoCalculo,descricaoInstalacaoEletrica,idEquipamento,descricaoEquipamento,GrupoCalculoId,Nome
0,IE_TR_ACES,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
1,IE_TR_AMB,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
2,IE_TR_ATV,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
3,IE_TR_BCH,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
4,IE_TR_CMT,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
5,IE_TR_OLEO,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
6,IE_TR_OUT,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
7,IE_TR_RESF,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
8,IE_TR_TQP,Health Index do Subsistema,1.0,2023-05-30 02:47:34.493,Bom Jesus da Lapa 2,32,Transformador de Força Reserva 05T3,2,Transformador de Força
9,IE_TR_ACES,Health Index do Subsistema,1.0,2023-05-30 02:45:38.637,Bom Jesus da Lapa 2,30,Transformador de Força 05T3 Fase B,2,Transformador de Força
